<a href="https://colab.research.google.com/github/AlbertCoding42/AI-Projects/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving albert_qa_dataset.csv to albert_qa_dataset.csv


In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer

# Load dataset
df = pd.read_csv("albert_qa_dataset.csv")

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

# Tokenize questions and answers
questions = [tokenizer.encode(question, return_tensors="pt") for question in df["Question"]]
answers = [tokenizer.encode(answer, return_tensors="pt") for answer in df["Answer"]]


In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")


In [ ]:
from transformers import AdamW
from tqdm.notebook import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

# Combine questions and answers into single sequences
combined_sequences = [tokenizer.encode(question + ' ' + answer, return_tensors="pt") for question, answer in zip(df["Question"], df["Answer"])]

EPOCHS = 3

for epoch in range(EPOCHS):
    total_loss = 0
    for sequence in tqdm(combined_sequences, total=len(combined_sequences)):
        optimizer.zero_grad()

        # Using the combined sequence as both input and labels
        outputs = model(input_ids=sequence, labels=sequence)

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{EPOCHS} Loss: {total_loss/len(combined_sequences)}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/28 [00:00<?, ?it/s]

Epoch 1/3 Loss: 3.3924263545445035


  0%|          | 0/28 [00:00<?, ?it/s]

Epoch 2/3 Loss: 1.5852405641760146


  0%|          | 0/28 [00:00<?, ?it/s]

Epoch 3/3 Loss: 0.8462397647755486


In [ ]:
model.save_pretrained("fine_tuned_model")

# Downloading
files.download("fine_tuned_model/config.json")
files.download("fine_tuned_model/pytorch_model.bin")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
torch.save(model.state_dict(), "fine_tuned_model_weights.pt")
files.download("fine_tuned_model_weights.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>